In [26]:
import os

from utils_park import *
from vocab import *

from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer

In [8]:
rq1 = '../newsbt_data/'
VOCAB_SIZE = 40000 # 40000
out_path = os.path.join(rq1, '40000_vocab_park/')

PAD = '<pad>'
UNK = '<unk>'
SOS = '<start>'
EOS = '<end>'

PAD_ID = 0
UNK_ID = 1
SOS_ID = 2
EOS_ID = 3

train_code_path = os.path.join(rq1, 'train/train.token.code')
train_type_path = os.path.join(rq1,'train/type')
train_simsbtcode_path = os.path.join(rq1,'train/train.token.simsbtcode')

valid_code_path = os.path.join(rq1, 'valid/valid.token.code')
valid_type_path = os.path.join(rq1,'valid/type')
valid_simsbtcode_path = os.path.join(rq1,'valid/valid.token.simsbtcode')

test_code_path = os.path.join(rq1, 'test/test.token.code')
test_type_path = os.path.join(rq1,'test/type')
test_simsbtcode_path = os.path.join(rq1,'test/test.token.simsbtcode')


In [9]:
train_code = read_file(train_code_path)
test_code = read_file(test_code_path)
valid_code = read_file(valid_code_path)

train_type = read_file(train_type_path)
test_type = read_file(test_type_path)
valid_type = read_file(valid_type_path)

In [15]:
def join_code_type(code_data, type_data):
    data = []
    for i, j in zip(code_data, type_data):
        temp = i + ' ' + j
        data.append(temp)
    return data


In [20]:
def join_code_type_withtoken(code_data, type_data):
    data = []
    for i, j in zip(code_data, type_data):
        temp = i + ' <code> ' + j
        data.append(temp)
    return data


In [22]:
train_data = join_code_type(train_code, train_type)
test_data = join_code_type(test_code, test_type)
valid_data = join_code_type(valid_code, valid_type)

train_data2 = join_code_type_withtoken(train_code, train_type)
test_data2 = join_code_type_withtoken(test_code, test_type)
valid_data2 = join_code_type_withtoken(valid_code, valid_type)

In [18]:
# out path
train_out_path = '../newsbt_data/train/'
test_out_path = '../newsbt_data/test/'
valid_out_path = '../newsbt_data/valid/'

In [23]:
save_file(train_out_path, "codetype", train_data)
save_file(test_out_path, "codetype", test_data)
save_file(valid_out_path, "codetype", valid_data)

save_file(train_out_path, "code_type", train_data2)
save_file(test_out_path, "code_type", test_data2)
save_file(valid_out_path, "code_type", valid_data2)

# Make Vocab

In [69]:
train_codetype_lines = read_file('../newsbt_data/train/codetype')
test_codetype_lines = read_file('../newsbt_data/test/codetype')
valid_codetype_lines = read_file('../newsbt_data/valid/codetype')
print(len(train_codetype_lines)) # 문장개수
print(len(test_codetype_lines)) # 문장개수
print(len(valid_codetype_lines)) # 문장개수

train_code_type_lines = read_file('../newsbt_data/train/code_type')
test_code_type_lines = read_file('../newsbt_data/test/code_type')
valid_code_type_lines = read_file('../newsbt_data/valid/code_type')
print(len(train_code_type_lines)) # 문장개수
print(len(test_code_type_lines)) # 문장개수
print(len(valid_code_type_lines)) # 문장/개수

445812
20000
20000
445812
20000
20000


In [77]:
t1 = Tokenizer(BPE())
t1.pre_tokenizer = Whitespace()
trainer = BpeTrainer(vocab_size=52000,
show_progress = True,
special_tokens = ["<pad>","<unk>", "<start>", "<end>"])
t1.train(files=["../newsbt_data/train/codetype"], trainer=trainer)

In [78]:
t1.save("../newsbt_data/vocabulary/codetype_bpe.json")

In [79]:
t2 = Tokenizer(BPE())
t2.pre_tokenizer = Whitespace()
trainer = BpeTrainer(vocab_size=52000,
show_progress = True,
special_tokens = ["<pad>","<unk>", "<start>", "<end>", "<code>"])
t2.train(files=["../newsbt_data/train/code_type"], trainer=trainer)

In [80]:
t2.save("../newsbt_data/vocabulary/code_type_bpe.json")

### indexing newsbtcode

In [81]:
my_tokenizer1 = Tokenizer.from_file("../newsbt_data/vocabulary/codetype_bpe.json")
my_tokenizer2 = Tokenizer.from_file("../newsbt_data/vocabulary/code_type_bpe.json")

In [82]:
def make_tokens(tokenizer, lines):
    data = []
    for i in lines:
        output = tokenizer.encode(i)
        data.append(output.ids)
    return data

In [83]:
# out path
train_out_path = '../newsbt_data/train/'
test_out_path = '../newsbt_data/test/'
valid_out_path = '../newsbt_data/valid/'

In [84]:
train_data1 = make_tokens(my_tokenizer1, train_codetype_lines)
test_data1 = make_tokens(my_tokenizer1, test_codetype_lines)
valid_data1 = make_tokens(my_tokenizer1, valid_codetype_lines)

train_data2 = make_tokens(my_tokenizer2, train_code_type_lines)
test_data2 = make_tokens(my_tokenizer2, test_code_type_lines)
valid_data2 = make_tokens(my_tokenizer2, valid_code_type_lines)

In [98]:
save_idx_file(train_out_path, "codetype_bpe", train_data1)
save_idx_file(test_out_path, "codetype_bpe", test_data1)
save_idx_file(valid_out_path, 'codetype_bpe', valid_data1)

save_idx_file(train_out_path, "code_type_bpe", train_data2)
save_idx_file(test_out_path, "code_type_bpe", test_data2)
save_idx_file(valid_out_path, 'code_type_bpe', valid_data2)
